<a href="https://colab.research.google.com/github/pinilDissanayaka/Simple-RAG/blob/main/Simple%20RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install a specific version (2.2.2) of the `sentence-transformers` package
!pip install sentence-transformers==2.2.2

# Install the `faiss-gpu` package for GPU-accelerated similarity search
!pip install faiss-gpu

# Install the `langchain` package, a library for building language model applications
!pip install langchain

# Install the `langchain_community` package, which includes community-contributed extensions for `langchain`
!pip install langchain_community

# Install the `langchain_groq` package, which adds support for Groq-based language model integration
!pip install langchain_groq

# Install the `langchain_huggingface` package to integrate `langchain` with Hugging Face models
!pip install langchain_huggingface

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.0.1
    Uninstalling sentence-transformers-3.0.1:
      Successfully uninstalled sentence-transformers-3.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.0.3 requires sentence-transformers>=2.6.0, but you have sentence-transformers 2.2.2 which is incompatible.
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [3]:
# Import the 'os' module for interacting with the operating system (e.g., file paths, environment variables)
import os

# Import the 'ChatGroq' class from the 'langchain_groq' module, which provides integration with Groq-based language models
from langchain_groq import ChatGroq

# Import the 'WebBaseLoader' class from 'langchain.document_loaders', used to load documents from web sources
from langchain.document_loaders.web_base import WebBaseLoader

# Import the 'HuggingFaceEmbeddings' class from 'langchain_huggingface', which provides embeddings using Hugging Face models
from langchain_huggingface import HuggingFaceEmbeddings

# Import the 'FAISS' class from 'langchain.vectorstores', which is used for vector similarity search with FAISS
from langchain.vectorstores import FAISS

# Import the 'RetrievalQA' class from 'langchain.chains', used to create a QA chain that retrieves relevant documents and answers questions
from langchain.chains import RetrievalQA

# Import the 'ChatPromptTemplate' class from 'langchain.prompts', used to define chat-based prompts for language models
from langchain.prompts import ChatPromptTemplate

# Import the 'RunnablePassthrough' class from 'langchain_core.runnables', which acts as a pass-through runnable for chaining components
from langchain_core.runnables import RunnablePassthrough

# Import the 'StrOutputParser' class from 'langchain_core.output_parsers', used to parse output as strings
from langchain_core.output_parsers import StrOutputParser

In [4]:
# Set an environment variable 'GROQ_API_KEY' using the value obtained from 'userdata'
from google.colab import userdata
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')

In [5]:
# Instantiate the 'ChatGroq' class to create a language model object

llm=ChatGroq(model="llama3-70b-8192",
             temperature=0.6)

In [6]:
# Use the 'load' method of 'WebBaseLoader' to fetch and parse the content from the specified URL
loader=WebBaseLoader("https://www.aljazeera.com/news/2024/8/12/paris-2024-olympics-close-los-angeles-takes-torch-five-key-takeaways")
data=loader.load()

print(data)

[Document(metadata={'source': 'https://www.aljazeera.com/news/2024/8/12/paris-2024-olympics-close-los-angeles-takes-torch-five-key-takeaways', 'title': 'Paris Olympics 2024 close, Los Angeles takes torch: Five key takeaways | Paris Olympics 2024 News | Al Jazeera', 'description': 'As the Paris Olympics ends with a spectacular closing ceremony, we take a look at some of the key moments.', 'language': 'en'}, page_content='Paris Olympics 2024 close, Los Angeles takes torch: Five key takeaways | Paris Olympics 2024 News | Al Jazeera\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip linksSkip to Contentplay Live Show navigation menuNavigation menuNewsShow more news sectionsMiddle EastAfricaAsiaUS & CanadaLatin AmericaEuropeAsia PacificWar on GazaUS ElectionOpinionSportVideoMoreShow more sectionsFeaturesUkraine warEconomyClimate CrisisInvestigationsInteractivesIn PicturesScience & TechnologyPodcastsplay Live Click here to searchsearchNews|Paris Olympics 2024Paris Olympics 2024 close, Los Ang

In [7]:
# Instantiate the 'HuggingFaceEmbeddings' class to create an embeddings model object
embeddingModel = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Create a vector database using the 'FAISS' class
# 'FAISS.from_documents' initializes the FAISS vector store from a list of documents
vectorDAtabase=FAISS.from_documents(data, embeddingModel)

In [9]:
# 'vectorDatabase.as_retriever()' creates a retriever that can be used to query the vector database
retriever=vectorDAtabase.as_retriever()

In [10]:
# Define a prompt template as a multi-line string
# This template is used to generate a prompt for the language model

template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


chatPromptTemplate = ChatPromptTemplate.from_template(template)

In [11]:
# Create a chain of components that processes a question based on the provided context

chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | chatPromptTemplate
    | llm
    | StrOutputParser()
)

In [12]:
# Invoke the processing chain with a specific question

chain.invoke("Who gave the Olympic flag to US gymnast Simone Biles?")

'LA Mayor Karen Bass gave the Olympic flag to US gymnast Simone Biles to hold as H E R sang the US national anthem live at the Stade de France.'